# Calculations of the impedance of a loudspeaker

Imports and function declarations

In [ ]:
import numpy as np
import pandas as pd
from scipy.fftpack import fft, ifft, fftfreq
import scipy.signal as signal
import matplotlib.pyplot as plt
from configparser import ConfigParser
dB2 = lambda x: 10*np.log10(np.abs(x))

Load the data and extract the frequency domain signals of the two voltages.

In [ ]:
folder = 'Single transducer'
folder = folder = '../Measurements/Impedance measurements/' + folder + '/'
config = ConfigParser()
config.read(folder + 'settings.ini')
fs=float(config['General']['fs'])
n=int(config['General']['blocksize'])
f=fftfreq(n,1/fs)
fLow = float(config['Signal']['Frequency'])
fHigh = float(config['Signal']['Parameter 2'])

timedata=pd.read_csv(folder + 'F1.txt',delimiter='\t')
frequencydata = timedata.apply(fft,raw=True)/n
spectrogram=timedata.apply(lambda x: signal.spectrogram(x,fs=fs,nperseg=2**15,scaling='spectrum'))


Use the circuit analysis of the resistor network to calculate the impedance of the loudspeaker.

In [ ]:
if any(frequencydata.columns == 'Black'):
    # Using the electroacoustics lab box
    # Balanced for approx 4 Ohm
    E_black = frequencydata['Black']
    E_red = frequencydata['Red']
    R1=0.38
    R2=12047
    R3=987
    Z=(R2+R3)*R1/R3 * E_black/E_red - R1
elif any(frequencydata.columns == 'Green'):
    # Using the prototype board circuit
    # Balanced for approx 220 Ohm
    E_green = frequencydata['Green']
    E_blue = frequencydata['Blue']
    R1=18.04
    R2=219.5e3
    R3=18e3
    Z=(R2+R3)*R1/R3 * E_blue/E_green - R1


Plot the result and calculate the nominal impedance.

In [ ]:
idx = (f>fLow) & (f<fHigh)
plt.semilogy(f[idx],np.abs(Z[idx]))
print('Minimum value in range is {:.1f} Ohm'.format(np.min(np.abs(Z[idx]))))
print('The nominal impedance is {:.0f} Ohm'.format(1.15*np.min(np.abs(Z[idx]))))
plt.show()
plt.plot(f[idx],np.unwrap(np.angle(Z[idx]))/np.pi)
#plt.xlim((fLow,fHigh))
plt.show()

Check that the amplifier output is good!

In [ ]:
spectrIdx='Red'
plt.pcolormesh(spectrogram[spectrIdx][1],
               spectrogram[spectrIdx][0],
               dB2(spectrogram[spectrIdx][2]),
              vmin=-100,vmax=None)
#plt.ylim((fLow,fHigh))
plt.colorbar()
plt.show()